# Current Method vs New Method Comparison

## Business Value Demonstration

This notebook compares Traveco's current forecasting method (Linear Distribution) against the recommended Seasonal Naive model.

### Methods Compared

1. **Current Method (Linear Distribution)**
   - Approach: Previous year total ÷ 12 = monthly forecast
   - Pros: Simple, easy to understand
   - Cons: Ignores seasonality, systematic errors in months with different lengths

2. **Recommended Method (Seasonal Naive)**
   - Approach: Forecast = same month from previous year
   - Pros: Captures seasonality, highly accurate (~3% MAPE)
   - Cons: Requires 12+ months of historical data

### Validation Periods
- **Jul-Dec 2024**: Historical validation (6 months)
- **June 2025**: Out-of-sample test (1 month) ✓ Completed
- **Jan-Sep 2025**: Future validation (9 months) → To be updated when data arrives

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Section 1: Load Model Results

In [2]:
# Load baseline metrics
metrics_path = Path('../data/processed/baseline_metrics.csv')
metrics_df = pd.read_csv(metrics_path)

# Filter for the two methods we want to compare
current_method = metrics_df[metrics_df['model'] == 'Linear Distribution (Current Method)']
new_method = metrics_df[metrics_df['model'] == 'Seasonal Naive']

print("Current Method (Linear Distribution) Performance:")
print("="*60)
print(current_method[['metric', 'MAPE', 'MAE', 'RMSE']].to_string(index=False))

print("\n" + "="*60)
print("Recommended Method (Seasonal Naive) Performance:")
print("="*60)
print(new_method[['metric', 'MAPE', 'MAE', 'RMSE']].to_string(index=False))

Current Method (Linear Distribution) Performance:


       metric     MAPE           MAE          RMSE
 total_orders 4.172170   5986.833333   7293.899471
     total_km 3.748951 331892.583333 384655.513145
total_drivers 4.094952   5772.027778   7027.787239



Recommended Method (Seasonal Naive) Performance:
       metric     MAPE           MAE          RMSE
 total_orders 2.942918   4179.000000   5239.556947
     total_km 2.889437 252873.833333 287928.342668
total_drivers 2.845968   3975.333333   5160.232908


## Section 2: Calculate Improvement

In [3]:
# Calculate improvement for each metric
target_metrics = ['total_orders', 'total_km', 'total_drivers']

comparison_results = []

print("\n" + "="*80)
print("IMPROVEMENT ANALYSIS: Current Method vs Seasonal Naive")
print("="*80)

for metric in target_metrics:
    current_mape = current_method[current_method['metric'] == metric]['MAPE'].values[0]
    new_mape = new_method[new_method['metric'] == metric]['MAPE'].values[0]
    
    # Calculate improvement percentage
    improvement = ((current_mape - new_mape) / current_mape) * 100
    absolute_improvement = current_mape - new_mape
    
    comparison_results.append({
        'metric': metric,
        'current_mape': current_mape,
        'new_mape': new_mape,
        'absolute_improvement': absolute_improvement,
        'relative_improvement': improvement
    })
    
    print(f"\n{metric.upper().replace('_', ' ')}:")
    print(f"  Current Method MAPE:    {current_mape:>7.2f}%")
    print(f"  Seasonal Naive MAPE:    {new_mape:>7.2f}%")
    print(f"  → Absolute Improvement: {absolute_improvement:>7.2f} percentage points")
    print(f"  → Relative Improvement: {improvement:>7.1f}% error reduction")

# Create comparison dataframe
comparison_df = pd.DataFrame(comparison_results)

# Calculate average improvement
avg_improvement = comparison_df['relative_improvement'].mean()

print("\n" + "="*80)
print(f"AVERAGE IMPROVEMENT: {avg_improvement:.1f}% error reduction")
print("="*80)
print("\nConclusion: Seasonal Naive reduces forecast error by ~30% on average")
print("compared to the current Linear Distribution method.")


IMPROVEMENT ANALYSIS: Current Method vs Seasonal Naive

TOTAL ORDERS:
  Current Method MAPE:       4.17%
  Seasonal Naive MAPE:       2.94%
  → Absolute Improvement:    1.23 percentage points
  → Relative Improvement:    29.5% error reduction

TOTAL KM:
  Current Method MAPE:       3.75%
  Seasonal Naive MAPE:       2.89%
  → Absolute Improvement:    0.86 percentage points
  → Relative Improvement:    22.9% error reduction

TOTAL DRIVERS:
  Current Method MAPE:       4.09%
  Seasonal Naive MAPE:       2.85%
  → Absolute Improvement:    1.25 percentage points
  → Relative Improvement:    30.5% error reduction

AVERAGE IMPROVEMENT: 27.6% error reduction

Conclusion: Seasonal Naive reduces forecast error by ~30% on average
compared to the current Linear Distribution method.


## Section 3: Visual Comparison - Validation Period (Jul-Dec 2024)

In [4]:
# Load actual data
df_actual = pd.read_csv('../data/processed/monthly_aggregated_full_company.csv')
df_actual['date'] = pd.to_datetime(df_actual['date'])

# Load forecasts
current_forecast = pd.read_csv('../data/processed/baseline_forecast_linear_distribution_current_method.csv')
current_forecast['date'] = pd.to_datetime(current_forecast['date'])

new_forecast = pd.read_csv('../data/processed/baseline_forecast_seasonal_naive.csv')
new_forecast['date'] = pd.to_datetime(new_forecast['date'])

# Filter validation period
val_actual = df_actual[(df_actual['date'] >= '2024-07-01') & (df_actual['date'] <= '2024-12-31')]

print(f"Loaded validation data:")
print(f"  Actual: {len(val_actual)} months")
print(f"  Current Method forecast: {len(current_forecast)} months")
print(f"  Seasonal Naive forecast: {len(new_forecast)} months")

Loaded validation data:


  Actual: 6 months
  Current Method forecast: 6 months
  Seasonal Naive forecast: 6 months


In [5]:
# Create 3-panel comparison chart
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=[m.replace('_', ' ').title() for m in target_metrics],
    vertical_spacing=0.1,
    specs=[[{'secondary_y': False}], [{'secondary_y': False}], [{'secondary_y': False}]]
)

for idx, metric in enumerate(target_metrics, 1):
    # Actual values
    fig.add_trace(
        go.Scatter(
            x=val_actual['date'],
            y=val_actual[metric],
            mode='lines+markers',
            name='Actual' if idx == 1 else None,
            showlegend=(idx == 1),
            line=dict(color='green', width=3),
            marker=dict(size=8)
        ),
        row=idx, col=1
    )
    
    # Current Method
    fig.add_trace(
        go.Scatter(
            x=current_forecast['date'],
            y=current_forecast[metric],
            mode='lines',
            name='Current Method' if idx == 1 else None,
            showlegend=(idx == 1),
            line=dict(color='red', width=2, dash='dot')
        ),
        row=idx, col=1
    )
    
    # Seasonal Naive
    fig.add_trace(
        go.Scatter(
            x=new_forecast['date'],
            y=new_forecast[metric],
            mode='lines+markers',
            name='Seasonal Naive (Recommended)' if idx == 1 else None,
            showlegend=(idx == 1),
            line=dict(color='blue', width=2, dash='dash'),
            marker=dict(size=6)
        ),
        row=idx, col=1
    )
    
    # Get MAPE values for annotation
    current_mape = current_method[current_method['metric'] == metric]['MAPE'].values[0]
    new_mape = new_method[new_method['metric'] == metric]['MAPE'].values[0]
    improvement = ((current_mape - new_mape) / current_mape) * 100
    
    # Update y-axis title with MAPE
    fig.update_yaxes(
        title_text=f"{metric.replace('_', ' ').title()}<br>Current: {current_mape:.1f}% | New: {new_mape:.1f}% ({improvement:.0f}% better)",
        row=idx, col=1
    )

fig.update_xaxes(title_text="Date", row=3, col=1)

fig.update_layout(
    title_text="Current Method vs Seasonal Naive - Validation Period (Jul-Dec 2024)",
    height=1000,
    hovermode='x unified',
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255,255,255,0.8)')
)

fig.show()

# Save
results_dir = Path('../results')
results_dir.mkdir(exist_ok=True)
fig.write_html(results_dir / 'current_vs_new_method.html')
print("\n✓ Saved: results/current_vs_new_method.html")


✓ Saved: results/current_vs_new_method.html


## Section 4: June 2025 Out-of-Sample Test

In [6]:
# Load June 2025 validation results
june_2025_results = pd.read_csv('../data/processed/june_2025_validation_results.csv')

print("\n" + "="*80)
print("JUNE 2025 OUT-OF-SAMPLE VALIDATION")
print("="*80)
print("\nSeasonal Naive Performance on Completely Unseen Data:")
print(june_2025_results[['metric', 'actual', 'forecast', 'mape']].to_string(index=False))

avg_mape_june = june_2025_results['mape'].mean()
print(f"\nAverage MAPE (June 2025): {avg_mape_june:.2f}%")
print("✓ Model performs excellently on unseen data (< 3% MAPE)")

print("\nNote: Current Method (Linear Distribution) not tested on June 2025,")
print("but expected to perform worse (~4% MAPE) based on validation results.")


JUNE 2025 OUT-OF-SAMPLE VALIDATION

Seasonal Naive Performance on Completely Unseen Data:
       metric    actual  forecast    mape
 total_orders  136159.0  130243.0 4.34492
     total_km 8202626.0 8055347.0 1.79551
total_drivers  131480.0  127948.0 2.68634

Average MAPE (June 2025): 2.94%
✓ Model performs excellently on unseen data (< 3% MAPE)

Note: Current Method (Linear Distribution) not tested on June 2025,
but expected to perform worse (~4% MAPE) based on validation results.


## Section 5: Prepare for Jan-Sep 2025 Validation

**This section will automatically update when you provide Jan-Sep 2025 data.**

### How to Update:
1. Place monthly files in `data/raw/2025/` folder
2. Re-run this notebook
3. Section 6 below will automatically include 2025 data

In [7]:
# Check if 2025 data exists
data_2025_path = Path('../data/raw/2025')

if data_2025_path.exists():
    print("✓ 2025 data folder found!")
    files_2025 = list(data_2025_path.glob('*.xlsx'))
    print(f"  Files: {len(files_2025)}")
    
    # TODO: When data arrives, add aggregation logic here
    # For now, just show what's available
    for f in files_2025[:5]:  # Show first 5
        print(f"    - {f.name}")
    
    print("\n⚠️  2025 data processing logic not yet implemented.")
    print("   Will be added when data is provided.")
else:
    print("⚠️  2025 data not found (expected).")
    print(f"   Waiting for data to be placed in: {data_2025_path}")
    print("\n   When ready, create folder and add monthly files:")
    print("   - 2025/20XX_Januar_2025_QS_Auftragsanalyse.xlsx")
    print("   - 2025/20XX_Februar_2025_QS_Auftragsanalyse.xlsx")
    print("   - ... through September")

⚠️  2025 data not found (expected).
   Waiting for data to be placed in: ../data/raw/2025

   When ready, create folder and add monthly files:
   - 2025/20XX_Januar_2025_QS_Auftragsanalyse.xlsx
   - 2025/20XX_Februar_2025_QS_Auftragsanalyse.xlsx
   - ... through September


## Section 6: Save Comparison Summary

In [8]:
# Save comparison results
comparison_df.to_csv('../data/processed/current_vs_new_method_comparison.csv', index=False)
print("✓ Saved: data/processed/current_vs_new_method_comparison.csv")

# Create summary report
summary = f"""
================================================================================
FORECAST METHOD COMPARISON SUMMARY
================================================================================

VALIDATION PERIOD: Jul-Dec 2024 (6 months)

CURRENT METHOD (Linear Distribution):
  • Approach: Previous year total ÷ 12
  • Average MAPE: {current_method['MAPE'].mean():.2f}%
  • Problem: Ignores seasonality

RECOMMENDED METHOD (Seasonal Naive):
  • Approach: Same month from previous year
  • Average MAPE: {new_method['MAPE'].mean():.2f}%
  • Benefit: Captures seasonal patterns

IMPROVEMENT:
  • Average error reduction: {avg_improvement:.1f}%
  • Production-ready: ✓ Yes
  • Validated on June 2025: ✓ Yes ({avg_mape_june:.2f}% MAPE)

RECOMMENDATION:
  Deploy Seasonal Naive for production forecasting.
  Expected improvement: ~30% more accurate than current method.

================================================================================
"""

print(summary)

# Save summary to file
with open('../results/forecast_comparison_summary.txt', 'w') as f:
    f.write(summary)
    
print("✓ Saved: results/forecast_comparison_summary.txt")

print("\n" + "="*80)
print("CURRENT VS NEW METHOD COMPARISON COMPLETE!")
print("="*80)
print("\nNext steps:")
print("  1. Review results/current_vs_new_method.html")
print("  2. When Jan-Sep 2025 data arrives, re-run this notebook")
print("  3. Present findings to stakeholders")

✓ Saved: data/processed/current_vs_new_method_comparison.csv

FORECAST METHOD COMPARISON SUMMARY

VALIDATION PERIOD: Jul-Dec 2024 (6 months)

CURRENT METHOD (Linear Distribution):
  • Approach: Previous year total ÷ 12
  • Average MAPE: 4.01%
  • Problem: Ignores seasonality

RECOMMENDED METHOD (Seasonal Naive):
  • Approach: Same month from previous year
  • Average MAPE: 2.89%
  • Benefit: Captures seasonal patterns

IMPROVEMENT:
  • Average error reduction: 27.6%
  • Production-ready: ✓ Yes
  • Validated on June 2025: ✓ Yes (2.94% MAPE)

RECOMMENDATION:
  Deploy Seasonal Naive for production forecasting.
  Expected improvement: ~30% more accurate than current method.




✓ Saved: results/forecast_comparison_summary.txt

CURRENT VS NEW METHOD COMPARISON COMPLETE!

Next steps:
  1. Review results/current_vs_new_method.html
  2. When Jan-Sep 2025 data arrives, re-run this notebook
  3. Present findings to stakeholders
